Übung : Einfaches Image-Management
-------------------------------------------------------

Wir wollen ein Image mit unseren eigene Usernamen auf https://hub.docker.com/ ablegen.

Dazu sind folgende Schritte notwendig:
* Erstellen eines Accounts auf https://hub.docker.com/.
* Username und Password als Umgebungsveriablen setzen (als Python Script)
* `docker login` ausführen
* Container Image `taggen` und auf https://hub.docker.com/ pushen


In [ ]:
import os
name = input("Username: ")
os.environ["USER"] = name

from getpass import getpass
password = getpass("Password: ")
os.environ["PW"] = password

In [ ]:
! docker login -u ${USER} -p ${PW}
! docker tag hello-world ${USER}/hello-world
! docker push ${USER}/hello-world

- - -

### Container Image im `tar` Format speichern

Nicht immer steht einen Registry zur Verfügung.

Alternativ kann man Container Images auch von/nach dem [tar](https://wiki.ubuntuusers.de/tar/) Format speichern und laden

In [ ]:
! docker save hello-world -o hello-world.tar
! docker rmi hello-world
! docker image ls

In [ ]:
! ls -ls *.tar

In [ ]:
! docker load -i hello-world.tar
! docker image ls